In [1]:
# from glob import glob
# import datetime as dt
import os
os.chdir('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow/')

In [2]:
import xarray as xr
import json
import pandas as pd
import numpy as np
from pandas.tseries.offsets import DateOffset
import useful_functions as uf

In [3]:
params = json.load(open(
    os.path.join('/g/data/vf71/la6889/lme_scale_calibration_ISMIP3a/new_workflow',
                 'outputs/dbpm_size_params.json')))

In [4]:
dbpm_input = pd.read_parquet(
    os.path.join('/g/data/vf71/la6889/dbpm_inputs/east_antarctica/monthly_weighted',
                 'dbpm_clim-fish-inputs_fao-58_1841-2010.parquet'))

## Parameters from `sizemodel` function

In [5]:
ERSEM_det_input = False
temp_effect = True
use_init = False

In [6]:
# Input parameters to vary:
# time series of intercept of plankton size spectrum (estimated from GCM, 
# biogeophysical model output or satellite data).		
ui0 = 10**np.array(params['int_phy_zoo'])
[numb_size_bins] = params['numb_size_bins']
#Size bin index
size_bin_index = np.arange(0, numb_size_bins)
#Size bins
log10_size_bins = np.array(params['log10_size_bins'])
size_bin_vals = 10**log10_size_bins
[log10_pred_prey_ratio] = params['log10_pred_prey_ratio']
[log_prey_pref] = params['log_prey_pref']
[metabolic_req_pred] = params['metabolic_req_pred']
[metabolic_req_detritivore] = params['metabolic_req_detritivore']
#Index for minimum detritivore size
[ind_min_detritivore_size] = params['ind_min_detritivore_size']
#Index for minimum predator size
[ind_min_pred_size] = params['ind_min_pred_size']
#Index for minimum size of detritivore fished
[ind_min_fish_det] = params['ind_min_fish_det']
#Index for minimum size of predator fished
[ind_min_fish_pred] = params['ind_min_fish_pred']
#Time
time = pd.date_range(start = min(dbpm_input.time)-DateOffset(months = 1),
                     end = max(dbpm_input.time), freq = 'MS')

# Initialising variables to store DBPM outputs

In [7]:
#data arrays for recording the two size spectra (V - det, U - pred)
predators = uf.init_da(log10_size_bins, time)
detritivores = uf.init_da(log10_size_bins, time)

#vector to hold detritus biomass density (g.m-2)
detritus = xr.DataArray(data = np.zeros(len(time)),
                        dims = 'time',
                        coords = {'time': time})

#data arrays for keeping track of growth (GG_v, GG_u) and reproduction (R_v, R_u) 
#from ingested food:
reprod_det = uf.init_da(log10_size_bins, time)
reprod_pred = uf.init_da(log10_size_bins, time)
growth_det = uf.init_da(log10_size_bins, time)
growth_int_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of predation mortality (PM_v, PM_u)
pred_mort_det = uf.init_da(log10_size_bins, time)
pred_mort_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of catches (Y_v, Y_u)
catch_det = uf.init_da(log10_size_bins, time)
catch_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of total mortality (Z_v, Z_u)
tot_mort_det = uf.init_da(log10_size_bins, time)
tot_mort_pred = uf.init_da(log10_size_bins, time)

#data arrays to hold fishing mortality rates at each size class at time 
#(Fvec_v, Fvec_u)
fishing_mort_det = uf.init_da(log10_size_bins, time)
fishing_mort_pred = uf.init_da(log10_size_bins, time)

#data arrays for keeping track of senescence mortality (SM_v, SM_u) and other 
#(intrinsic) mortality (OM_v, OM_u)
senes_mort_det = np.zeros(len(log10_size_bins))
senes_mort_pred = np.zeros(len(log10_size_bins))
other_mort_det = np.zeros(len(log10_size_bins))
other_mort_pred = np.zeros(len(log10_size_bins))

# Building lookup tables

In [8]:
#lookup tables for terms in the integrals which remain constant over time (gphi, mphi)
constant_growth = uf.gphi_f(uf.pred_prey_matrix(log10_size_bins), log10_pred_prey_ratio, 
                            log_prey_pref)
constant_mortality = uf.mphi_f(-uf.pred_prey_matrix(log10_size_bins), log10_pred_prey_ratio, 
                               log_prey_pref, metabolic_req_pred)

#lookup table for components of 10^(metabolic_req_pred*log10_size_bins) (expax)
met_req_log10_size_bins = uf.expax_f(log10_size_bins, metabolic_req_pred)

# Numerical integration

In [9]:
# set up with the initial values from param - same for all grid cells
#(phyto+zoo)plankton size spectrum (U) 
predators.isel(size_class = slice(None, 120)).loc[{'time': predators.time.min()}] = \
params['plank_pred_sizes'][:120]

# set initial detritivore spectrum (V)
detritivores.isel(size_class = slice((ind_min_detritivore_size-1), 120)).\
loc[{'time': detritivores.time.min()}] = \
np.array(params['detritivore_sizes'])[ind_min_detritivore_size-1:120]

# set initial detritus biomass density (g.m^-3) (W)
detritus.loc[{'time': detritus.time.min()}] = params['init_detritus'][0]

In [10]:
if use_init:
    # set up with the initial values from previous run
    predators.isel(size_class = slice((ind_min_pred_size-1), None)).\
    loc[{'time': predators.time.min()}] = \
    params['plank_pred_sizes'][(ind_min_pred_size-1):]
    
    # set initial detritivore spectrum from previous run
    detritivores.isel(size_class = slice((ind_min_detritivore_size-1), None)).\
    loc[{'time': detritivores.time.min()}] = \
    np.array(params['detritivore_sizes'])[(ind_min_detritivore_size-1):]

In [11]:
#intrinsic natural mortality (OM.u, OM.v)
other_mort_det = params['natural_mort']*10**(-0.25*log10_size_bins)
other_mort_pred = params['natural_mort']*10**(-0.25*log10_size_bins)

#senescence mortality rate to limit large fish from building up in the system
#same function as in Law et al 2008, with chosen parameters gives similar M2 values
#as in Hall et al. 2006 (SM.u, SM.v)
senes_mort_det = (params['const_senescence_mort']*
                  10**(params['exp_senescence_mort']*
                       (log10_size_bins - params['size_senescence'])))

senes_mort_pred = (params['const_senescence_mort']*
                  10**(params['exp_senescence_mort']*
                       (log10_size_bins - params['size_senescence'])))

## Fishing mortality (FVec.u, FVec.v)
From Benoit & Rochet 2004. Here `fish_mort_pred` and `fish_mort_pred` equal fixed catchability term for predators and detritivores. To be estimated along with `ind_min_det` and `ind_min_fish_pred`

In [12]:
fishing_mort_pred.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).\
loc[{'time': fishing_mort_pred.time.min()}] = (params['fish_mort_pred']*
                                               np.array(params['effort'][0]))

fishing_mort_det.isel(size_class = slice(int(ind_min_fish_det-1), -1)).\
loc[{'time': fishing_mort_det.time.min()}] = (params['fish_mort_detritivore']*
                                              np.array(params['effort'][0]))

In [13]:
#output fisheries catches per yr at size - predators (Y_u)
catch_pred.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).\
loc[{'time': catch_pred.time.min()}] = \
(fishing_mort_pred.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).
 sel(time = fishing_mort_pred.time.min())*
 predators.isel(size_class = slice(int(ind_min_fish_pred-1), -1)).
 sel(time = predators.time.min())*size_bin_vals[int(ind_min_fish_pred-1): -1])

#output fisheries catches per yr at size - detritivores (Y_v)
catch_det.isel(size_class = slice(int(ind_min_fish_det-1), -1)).\
loc[{'time': catch_pred.time.min()}] = \
(fishing_mort_det.isel(size_class = slice(int(ind_min_fish_det-1), -1)).
 sel(time = fishing_mort_pred.time.min())*
 detritivores.isel(size_class = slice(int(ind_min_fish_det-1), -1)).
 sel(time = predators.time.min())*size_bin_vals[int(ind_min_fish_det-1): -1])

In [14]:
if temp_effect:
    #Adding time dimension to temperature effect for pelagic group
    pel_tempeffect = xr.DataArray(data = np.exp(params['c1']-params['activation_energy']/
                                                (params['boltzmann']*
                                                 (np.array(params['sea_surf_temp'])+273))),
                                  dims = ['time'],
                                  coords = {'time': dbpm_input.time})
    
    #Adding time dimension to temperature effect for benthic group
    ben_tempeffect = xr.DataArray(data = np.exp(params['c1']-params['activation_energy']/
                                                (params['boltzmann']*
                                                 (np.array(params['sea_floor_temp'])+273))),
                                  dims = ['time'],
                                  coords = {'time': dbpm_input.time})
else:
    pel_tempeffect = 1
    ben_tempeffect = 1

#To be applied to feeding rates for pelagics and benthic groups
feed_multiplier = params['hr_volume_search']*10**(log10_size_bins*metabolic_req_pred)

# Ingested food
growth_prop = 1-np.array(params['defecate_prop'])
# High quality food
high_prop = 1-np.array(params['def_low'])

/jobfs/128677042.gadi-pbs/ipykernel_532897/1272663677.py:3: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the DataArray or Variable constructor; it can be silenced by converting the values to nanosecond precision ahead of time.
  pel_tempeffect = xr.DataArray(data = np.exp(params['c1']-params['activation_energy']/
/jobfs/128677042.gadi-pbs/ipykernel_532897/1272663677.py:10: UserWarning: Converting non-nanosecond precision datetime values to nanosecond precision. This behavior can eventually be relaxed in xarray, as it is an artifact from pandas which is now beginning to support non-nanosecond precision values. This warning is caused by passing non-nanosecond np.datetime64 or np.timedelta64 values to the D

## Start of loop along time dimension

In [15]:
# for i, t in enumerate(dbpm_input.time):
#     print(i, t)
i = 0
t = dbpm_input.time[0]
ts = time[i]

### Calculate growth and mortality

In [16]:
# feeding rates pelagics yr-1 (f_pel)
pred_growth = ((predators.sel(time = ts)*
                params['log_size_increase']).values@constant_growth)
feed_rate_pel = (pel_tempeffect.sel(time = t).values*\
                 (((feed_multiplier*params['pref_pelagic'])*pred_growth)/
                  (1+params['handling']*((feed_multiplier*params['pref_pelagic'])
                                         *pred_growth))))

# feeding rates benthics yr-1 (f_ben)
detrit_growth = ((detritivores.sel(time = ts)*
                  params['log_size_increase']).values@constant_growth)
feed_rate_bent = (pel_tempeffect.sel(time = t).values*\
                 (((feed_multiplier*params['pref_benthos'])*detrit_growth)/
                  (1+params['handling']*((feed_multiplier*params['pref_benthos'])
                                         *detrit_growth))))

# feeding rates detritivores yr-1 (f_det)
detritus_multiplier = ((1/size_bin_vals)*params['hr_vol_filter_benthos']*
                       (10**(log10_size_bins*metabolic_req_detritivore)*
                       detritus.sel(time = ts).values))
feed_rate_det = (ben_tempeffect.sel(time = t).values*detritus_multiplier/
                 (1+params['handling']*detritus_multiplier))

# Predator growth integral yr-1 (GG_u)
growth_int_pred.loc[{'time': ts}] = (growth_prop*np.array(params['growth_pred'])*
                                     feed_rate_pel+high_prop*
                                     np.array(params['growth_detritivore'])*feed_rate_bent)

# Reproduction yr-1 (R_u)
if params['dynamic_reproduction']:
    reprod_pred.loc[{'time': ts}] = (growth_prop*(1-(np.array(params['growth_pred'])+
                                                     params['energy_pred']))*
                                     feed_rate_pel+growth_prop*
                                     (1-(np.array(params['growth_detritivore'])+
                                         params['energy_detritivore']))*feed_rate_bent)

# Predator death integrals 
#Satiation level of predator for pelagic prey
divisor = feed_multiplier*params['pref_pelagic']*pred_growth
sat_pel = [fp/fpt if fp > 0 else 0 for fp, fpt in zip(feed_rate_pel, divisor)]

# yr-1 (PM_u)
pred_mort_pred.loc[{'time': ts}] = (np.array(params['pref_pelagic'])*
                                    params['hr_volume_search']*met_req_log10_size_bins*
                                   ((predators.sel(time = ts)*sat_pel*
                                    params['log_size_increase']).values@constant_mortality))

# yr-1 (Z_u)
tot_mort_pred.loc[{'time': ts}] = (pred_mort_pred.sel(time = ts)+
                                   pel_tempeffect.sel(time = t).values*
                                   other_mort_pred+senes_mort_pred+
                                   fishing_mort_pred.sel(time = ts))

# Benthos growth integral yr-1 (GG_v)
growth_det.loc[{'time': ts}] = (np.array(high_prop)*params['growth_detritus']*
                                feed_rate_det)

#reproduction yr-1 (R_v)
if params['dynamic_reproduction']:
    reprod_det.loc[{'time': ts}] = (high_prop*(1-(np.array(params['growth_detritus'])+
                                              params['energy_detritivore']))*
                                feed_rate_det)

# Benthos death integral
# Satiation level of predator for benthic prey 
divisor = (params['hr_volume_search']*10**(log10_size_bins*
                                           metabolic_req_detritivore)*
            params['pref_benthos']*detrit_growth)
sat_ben = [fb/fpt if fb > 0 else 0 for fb, fpt in zip(feed_rate_bent, divisor)]

# yr-1 (PM_v)
new_mort = ((np.array(params['pref_benthos'])*params['hr_volume_search']*
             met_req_log10_size_bins)*
            ((predators.sel(time = ts)*sat_ben*
              params['log_size_increase']).values@constant_mortality))
pred_mort_det.loc[{'time': ts}] = [nm if sb > 0 else 0 for sb, nm in zip(sat_ben, new_mort)]

# yr-1 (Z_v)
tot_mort_det.loc[{'time': ts}] = (pred_mort_det.sel(time = ts)+
                                  ben_tempeffect.sel(time = t).values*
                                  other_mort_det+senes_mort_det+
                                  fishing_mort_det.sel(time = ts))

#total biomass density eaten by pred (g.m-2.yr-1)
eatenbypred = (size_bin_vals*feed_rate_pel*predators.sel(time = ts)+size_bin_vals*
               feed_rate_bent*predators.sel(time = ts))

#detritus output (g.m-2.yr-1)
# losses from detritivore scavenging/filtering only:
output_w = (size_bin_vals*feed_rate_det*detritivores.sel(time = ts)*
            params['log_size_increase']).sum().values

#total biomass density defecated by pred (g.m-2.yr-1)
defbypred = (params['defecate_prop']*(feed_rate_pel)*size_bin_vals*
             predators.sel(time = ts)+params['def_low']*feed_rate_bent*
             size_bin_vals*predators.sel(time = ts))

# Increment values of detritus, predators & detritivores for next time step  
#Detritus Biomass Density Pool - fluxes in and out (g.m-2.yr-1) of 
#detritus pool and solve for detritus biomass density in next time step 
if not ERSEM_det_input:
    #considering pelagic faeces as input as well as dead bodies from both 
    #pelagic and benthic communities and phytodetritus (dying sinking
    #phytoplankton)
    if params['detritus_coupling']:
        # pelagic spectrum inputs (sinking dead bodies and faeces) - export 
        # ratio used for "sinking rate" + benthic spectrum inputs (dead stuff
        # already on/in seafloor)
        input_w = (params['sinking_rate'][i]* 
           ((defbypred.isel(size_class = slice(ind_min_pred_size-1, 
                                               numb_size_bins))*
             params['log_size_increase']).sum()+
            (pel_tempeffect.sel(time = t).values*other_mort_pred*
             predators.sel(time = ts)*size_bin_vals*
             params['log_size_increase']).sum()+ 
            (pel_tempeffect.sel(time = t).values*senes_mort_pred*
             predators.sel(time = ts)*size_bin_vals*
             params['log_size_increase']).sum())+
           ((ben_tempeffect.sel(time = t).values*other_mort_det*
             detritivores.sel(time = ts)*size_bin_vals*
             params['log_size_increase']).sum()+ 
            (ben_tempeffect.sel(time = t).values*senes_mort_det*
             detritivores.sel(time = ts)*size_bin_vals*
             params['log_size_increase']).sum()))
    else:
        input_w = ((ben_tempeffect.sel(time = t).values*other_mort_det*
                    detritivores.sel(time = ts)*size_bin_vals*
                    params['log_size_increase']).sum()+
                   (ben_tempeffect.sel(time = t).values*senes_mort_det*
                    detritivores.sel(time = ts)*size_bin_vals*
                    params['log_size_increase']).sum())

    # get burial rate from Dunne et al. 2007 equation 3
    burial = input_w*(0.013+0.53*input_w**2/(7+input_w)**2)

    # losses from detritivory + burial rate (not including remineralisation
    # bc that goes to p.p. after sediment, we are using realised p.p. as
    # inputs to the model) 
    dW = input_w-(output_w+burial)

    #biomass density of detritus g.m-2
    detritus.loc[{'time': t}] = ((detritus.sel(time = ts).values+
                                  dW.values*params['timesteps_years']).item())
else:
    detritus.loc[{'time': t}] = detritus.sel(time = ts)

# Pelagic Predator Density (nos.m-2)- solve for time + timesteps_years using
# implicit time Euler upwind finite difference (help from Ken Andersen 
# and Richard Law)

# Matrix setup for implicit differencing 
Ai_u = np.zeros(len(log10_size_bins))
Bi_u = np.zeros(len(log10_size_bins))
Si_u = np.zeros(len(log10_size_bins))

Ai_u[1:] = ((1/np.log(10))*
            -growth_int_pred.isel(size_class = slice(None, -1)).sel(time = ts)*
            params['timesteps_years']/params['log_size_increase'])
Bi_u[1:] = (1+(1/np.log(10))*
            growth_int_pred.isel(size_class = slice(1, None)).sel(time = ts)*
            params['timesteps_years']/params['log_size_increase']+
            tot_mort_pred.isel(size_class = slice(1, None)).sel(time = ts)*
            params['timesteps_years'])
Si_u[1:] = predators.isel(size_class = slice(1, None)).sel(time = ts)

# Boundary condition at upstream end 
Ai_u[(ind_min_pred_size-1)] = 0
Bi_u[(ind_min_pred_size-1)] = 1
Si_u[(ind_min_pred_size-1)] = predators.isel(size_class = (ind_min_pred_size-1)).\
sel(time = ts)

# Invert matrix
#recruitment at smallest consumer mass
#continuation of plankton hold constant  
if use_init:
    predators.isel(size_class = slice(None, ind_min_pred_size)).\
    loc[{'time': t}] = (ui0[i]*10**(params['slope_phy_zoo'][i]*
                                    log10_size_bins)[:ind_min_pred_size])
else:
    predators.isel(size_class = slice(None, ind_min_pred_size)).\
    loc[{'time': t}] = (ui0[i+1]*10**(params['slope_phy_zoo'][i+1]*
                                      log10_size_bins)[:ind_min_pred_size])

# apply transfer efficiency of 10% *plankton density at same size
# reproduction from energy allocation
if params['dynamic_reproduction']:
    predators.isel(size_class = ind_min_pred_size-1).\
    loc[{'time': t}] = (predators.isel(size_class = ind_min_pred_size-1).\
                        sel(time = ts).values+
                        ((reprod_pred.isel(size_class = slice(ind_min_pred_size,
                                                              None)).sel(time = ts)*
                          size_bin_vals[ind_min_pred_size:]*
                          predators.isel(size_class = slice(ind_min_pred_size, 
                                                            None)).sel(time = ts)*
                          params['log_size_increase']).sum().values*
                         params['timesteps_years'])/
                        (np.array(params['log_size_increase'])*
                         size_bin_vals[ind_min_pred_size-1])-
                        (np.array(params['timesteps_years'])/
                         params['log_size_increase'])*(1/np.log(10))*
                        (growth_int_pred.isel(size_class = ind_min_pred_size-1).\
                         sel(time = ts)*
                         predators.isel(size_class = ind_min_pred_size-1).\
                         sel(time = ts)).values-params['timesteps_years']*
                        ((tot_mort_pred.isel(size_class = ind_min_pred_size-1).\
                          sel(time = ts)*
                          predators.isel(size_class = ind_min_pred_size-1).\
                          sel(time = ts)).values)).item()

#main loop calculation
for j in range(ind_min_pred_size, numb_size_bins):
    predators.isel(size_class = j).\
    loc[{'time': t}] = ((Si_u[j]-Ai_u[j]*predators.isel(size_class = j-1).\
                         sel(time = t))/Bi_u[j])

# Benthic Detritivore Density (nos.m-2) 
Ai_v = np.zeros(len(log10_size_bins))
Bi_v = np.zeros(len(log10_size_bins))
Si_v = np.zeros(len(log10_size_bins))

#shorthand for matrix referencing
idx = np.arange(ind_min_detritivore_size, numb_size_bins)

Ai_v[idx] = (((1/np.log(10))*
              -growth_det.isel(size_class = slice(ind_min_detritivore_size-1, -1)).\
              sel(time = ts)).values*params['timesteps_years']/
             params['log_size_increase'])
Bi_v[idx] = (1+(1/np.log(10))*
             growth_det.isel(size_class = slice(ind_min_detritivore_size, None)).\
             sel(time = ts).values*params['timesteps_years']/
             params['log_size_increase']+
             tot_mort_det.isel(size_class = slice(ind_min_detritivore_size, None)).\
             sel(time = ts)*params['timesteps_years'])
Si_v[idx] = (detritivores.isel(size_class = slice(ind_min_detritivore_size, None)).\
             sel(time = ts))

#boundary condition at upstream end
Ai_v[ind_min_detritivore_size-1] = 0
Bi_v[ind_min_detritivore_size-1] = 1
Si_v[ind_min_detritivore_size-1] = (detritivores.\
                                    isel(size_class = ind_min_detritivore_size-1).\
                                    sel(time = ts))

#invert matrix
#recruitment at smallest detritivore mass  
#hold constant continuation of plankton with sinking rate multiplier 
(detritivores.isel(size_class = slice(None, 
                                     ind_min_detritivore_size)).\
 loc[{'time': t}]) = (detritivores.isel(size_class = slice(None,
                                                          ind_min_detritivore_size)).\
                                       sel(time = ts))

# apply a very low of transfer efficiency 1%* total biomass of detritus
#divided by minimum size
if params['dynamic_reproduction']:
    (detritivores.\
     isel(size_class = ind_min_detritivore_size-1).\
     loc[{'time': t}]) = 

# To be checked against `R` results

Can `params['sinking_rate']` be transformed to data frame? Time dimension should be added to keep track

In [303]:
(detritivores.\
 isel(size_class = ind_min_detritivore_size-1).\
 loc[{'time': t}])

<xarray.DataArray ()> Size: 8B
array(657.29901848)
Coordinates:
    size_class  float64 8B -3.0
    time        datetime64[ns] 8B 1841-01-01

In [361]:
# detritivores.isel(size_class = ind_min_detritivore_size-1).sel(time = ts)#+
# (((reprod_det.isel(size_class = slice(ind_min_detritivore_size, None)).sel(time = ts)*
# size_bin_vals[ind_min_detritivore_size:]*
# detritivores.isel(size_class = slice(ind_min_detritivore_size, None)).sel(time = ts)).values*
# params['log_size_increase']).sum()*np.array(params['timesteps_years'])/
((np.array(params['log_size_increase'])*size_bin_vals[ind_min_detritivore_size-1])-
(np.array(params['timesteps_years'])/params['log_size_increase'])*(1/np.log(10))*
(growth_det.isel(size_class = ind_min_detritivore_size-1).sel(time = ts)*
detritivores.isel(size_class = ind_min_detritivore_size-1).sel(time = ts)).values- 
(params['timesteps_years']*
(tot_mort_det.isel(size_class = ind_min_detritivore_size-1).sel(time = ts)*
detritivores.isel(size_class = ind_min_detritivore_size-1).sel(time = ts)).values))

array([-12.66142569])

In [309]:
idx

array([ 91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180])

In [210]:
ore